# Homework 2

## Exercise 1
Assume we have 1 dollar to be invested in two assets, whose return is modeled as a bivariate Gaussian distribution such that

$$
\Sigma = \begin{pmatrix}
0.25 & 0.15 \\ 0.15 & 0.10
\end{pmatrix}
$$

How much should we invest in each to minimize the overall variance in return with and without short selling? Answer this question solving the problem analytically and using a numerical code for constrained optimization problems. To simplify the problem, the acceptable baseline expected rate of return $m_b$ is $−\infty$.

### Analytical solution
In this problem we have to minimize the variance with inequality constrains. This variance is computed in the following way $f(\textbf{w}) = \textbf{w}^\top \Sigma\textbf{w}$ where each component of the vector $\textbf{w}$ is the weight that the corresponding asset is going to have in our portfolio.

#### With short selling

The constrains that we have to take into account are $\sum_i^nw_i=1$, this means that our weights must sum to 1. Now to solve the problem lets compute the function $f(\textbf{w})$.

$$
    f(\textbf{w}) = \begin{pmatrix} w_1 & w_2 \end{pmatrix} \begin{pmatrix} 0.25 & 0.15 \\ 0.15 & 0.10 \end{pmatrix}\begin{pmatrix} w_1 \\ w_2 \end{pmatrix}= 0.25 w_1^2+0.3w_1w_2+0.1w_2^2=f(w_1,w_2).
$$

With this function we have to solve a optimization problem in two dimensions but we can transform it by taking into account the constrain $w_1+w_2=1\to w_1=1-w_2$. So if we substitute this in the original equation we get $f(w_2) = 0.05w_2^2-0.2w_2+0.25,$ that is a function of only one variable so to find its minimum we just have to derive it.

$$
f'(w_2) = 0.1w_2-0.2.
$$

And the minimum is achieved at $0.1w_2 = 0.2\to w_2 = 2$. With this value of $w_2$ we get $w_1=-1$ and the solution is:

$$
    \textbf{w} = \begin{pmatrix}-1\\2\end{pmatrix}.
$$

With this solution we see that we have to short sell the first asset and then buy the second one.

#### Without short selling
In this case we don't allow the weights to be negative, but the rest of the problem is the same. We have to implement the following constrains $w_1\geq0$ and $w_2\geq0$. Because $w_2$ is positive and $w_1=1-w_2$ is also positive we have that $w_2$ must be also less or equal than 1, so $w_2\in[0,1]$. The absolute minimum from of the function $f(w_2)$, obtained in [the previous section](#with-short-selling), is not inside this region so because we know that this function only has an extrema point the minimum value inside the interval must be achieved at the boundaries.

$$
f(0) = 0.25 \;\;\;\text{and}\;\;\; f(1) = 0.1
$$

So the minimum is at $w_2=1$ therefore $w_1$ must be 0.

$$
    \textbf{w} = \begin{pmatrix}0\\1\end{pmatrix}.
$$

## Numeric solution
To find a numerical solution we are going to implement something similar to the Newton method that we use for the previous assignment. In order for the Newton method to work with equality constrains we have to choose as a starting point a feasible one and we have to redefine the step of the Newton method.

First we are going to define some functions that we are going to use, as the function itself, the gradient and the line search method. 

In [137]:
import numpy as np
import numpy.linalg as npl

def f(w):
    return 0.25*w[0]**2 + 0.30*w[0]*w[1] + 0.1*w[1]**2

def gradient(w):
    return np.array([0.5*w[0] + 0.3*w[1], 0.3*w[0] + 0.2*w[1]])

def line_search(d_k, x, t=1):
    """Implement the line search method, this function update the time step
    until the criterion is fulfilled"""

    alpha = 0.01
    beta = 0.5
    while f(x + t*d_k) > f(x) + alpha*t*np.dot(gradient(x),d_k):
        t *= beta
    return t

As in the previous assignment we need to compute the Hessian matrix, in this case this matrix is:

$$
H = \begin{pmatrix}
\partial^2_{w_1}f & \partial_{w_1}\partial_{w_2}f \\
\partial_{w_1}\partial_{w_2}f & \partial^2_{w_2}f
\end{pmatrix} = \begin{pmatrix}
0.5 & 0.3 \\ 0.3 & 0.2
\end{pmatrix} = \nabla^2f
$$

We have to redefine the step of this method and to do so we need to solve a different system that the one on the previous assignment. In this case the system that we have to solve is:

$$
\begin{bmatrix}
\nabla^2f & A^\top \\ A & 0
\end{bmatrix}
\begin{bmatrix} d_k \\ \lambda_k \end{bmatrix}
=
\begin{bmatrix} -\nabla f \\ 0 \end{bmatrix},
$$

where $A = \begin{pmatrix} 1 & 1 \end{pmatrix}$, is the matrix that gives us the constrain $A\text{w}=1$.

Now lets put the code for the newton method.

In [138]:
HESSIAN = np.array([[0.5, 0.3], [0.3, 0.2]])
A = np.array([[1.0, 1.0]])
# Define the matrix above
MATRIX = np.zeros((3, 3))
MATRIX[:2,:2] = HESSIAN
MATRIX[:2,-1:] = A.T
MATRIX[-1:,:2] = A

def newtons_method(stop, x, short_selling = False):
    """Implementation of the newton method"""

    grad = gradient(x)
    if np.dot(grad, grad) < stop**2:  # check the case that we are already in a minimum
        return x, 0

    for ite in range(1,1000):
        g = gradient(x)
        lhs = np.concatenate((-g.T,[0])).reshape((3,1))
        solution = npl.solve(MATRIX, lhs)
        descend_direction = solution[:2].reshape((2,))
        
        if short_selling:
            crit = np.dot(g, descend_direction)
            if np.abs(crit) < stop:
                return x, ite
            t = line_search(descend_direction, x)
            x = x + t * descend_direction
        else:
            t = 1
            while min(x + t*descend_direction) < 0:
                t *= 0.5
            t = line_search(descend_direction, x, t)
            x = x + t * descend_direction



    print("The stop criterium wasn't achieve in 1000 iterations.")
    return x, ite

### With short selling
In this case the solution *w* could have negative values.

In [139]:
newtons_method(1.0e-05, [0.5,0.5], short_selling=True)

(array([-1.,  2.]), 2)